The app will only take a few wines as input and will only try to predict one wine.

In [182]:
import pandas as pd
import numpy as np

from random import randrange
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from lightfm import LightFM

In [114]:
with open('data.csv') as file:
    data = pd.read_csv(file)

In [115]:
with open('users.csv') as file:
    users_df = pd.read_csv(file)
users = users_df.values

In [185]:
with open('test.csv') as file:
    test_df = pd.read_csv(file)
test = test_df.values

For demonstration purposes, pick a random user out of test, use one of the wines as the "Will I like this wine?", use five more wines as the "I like these wines", and see if the prediction matches or not.

In [186]:
rows, cols = test.shape

In [187]:
demo_user = test[:,randrange(cols)] #choose a random user from the test set

In [188]:
user_indexes = np.nonzero(demo_user)[0] #find the index of all known wines

In [189]:
type(user_indexes)

numpy.ndarray

In [190]:
user_indexes.shape

(143,)

In [191]:
unknown_wine = np.random.choice(user_indexes)#item_id for predict

In [192]:
user_indexes = user_indexes[user_indexes != unknown_wine]

In [193]:
user_indexes.shape

(142,)

In [194]:
known_wines = []
for i in range(5):
    known_wines.append(np.random.choice(user_indexes))

In [195]:
new_user = np.zeros_like(demo_user)
new_user[known_wines] = 1

In [196]:
users.shape

(6497, 4000)

In [197]:
new_users = np.column_stack((users, new_user))

In [198]:
num_items, num_users = new_users.shape #num_users will be the same as the user_id
print(num_items)
print(num_users)

6497
4001


Train the model with the new user.

In [199]:
users_sparse = coo_matrix(new_users.T)

In [200]:
users_sparse.shape

(4001, 6497)

In [201]:
item_features = data.values
item_features = np.delete(item_features, -2, axis=1)

In [202]:
item_features = csr_matrix(item_features, dtype=np.float32)

In [203]:
item_features.shape

(6497, 12)

In [204]:
model = LightFM(loss='warp')

In [206]:
model = model.fit(users_sparse, item_features=item_features)

In [207]:
prediction = model.predict(num_users-1, np.arange(num_items, dtype=np.int32), item_features=item_features)

In [208]:
prediction[prediction > 0] = 1

In [209]:
prediction[prediction <= 0 ] = 0

In [210]:
prediction[unknown_wine]

1.0